<p id = 'toc'></p>

<h1 style = "font-family:Verdana; font-size:24px; font-weight: bold; letter-spacing: 3px; color: teal;"> TABLE OF CONTENTS </h1>

---

[<h3 style = "font-family: Verdana; font-size: 18px; text-indent: 8vw; letter-spacing: 2px; color: teal">0&nbsp;&nbsp;&nbsp;&nbsp; IMPORTS </h3>](#0)

---

<h3 style = "font-family:Verdana; font-size: 18px; text-indent: 8vw; letter-spacing: 2px; color: teal"><a href='#1'>1.&nbsp;&nbsp;&nbsp;&nbsp;DATASET</a></h3>

---

<h3 style = "font-family:Verdana; font-size: 18px; text-indent: 8vw; letter-spacing: 2px; color: teal"><a href='#2'>2.&nbsp;&nbsp;&nbsp;&nbsp;CONVERTING IMAGE DATA TO JPG</a></h3>


<a id = '0'></a>
# <span><center style = "font-family: Verdana; font-size:36px; color : teal; letter-spacing: 3px; font-weight: bold">0. &nbsp;IMPORTS&nbsp;&nbsp;&nbsp;&nbsp;<a href = '#toc'>&#10514;</a></center></span>

In [ ]:
# pip install pydicom

In [ ]:
import pandas as pd 
import numpy as np
import os
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import torch
import cv2
import torchvision.transforms as transforms
import shutil


from PIL import Image
from tqdm import tqdm
from torchvision import datasets
from pydicom import dcmread

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False

In [ ]:
class config:
    TRAIN_PATH = '../input/unifesp-x-ray-body-part-classifier/train'
    TRAIN_CSV = '../input/unifesp-x-ray-body-part-classifier/train.csv'
    WORKING_PATH = './'
    TRAIN_JPG_PATH = '../input/unifesp-xray-jpg-dataset/train'
    RANDOM_SEED = 15

<a id="1"></a>

# <span><center style= "font-family:Verdana; font-size:36px; font-weight: bold; letter-spacing: 3px; color: teal">1.&nbsp;&nbsp;DATASET&nbsp;&nbsp;&nbsp;&nbsp;<a href = '#toc'>&#10514;</a> </center></span>

In [ ]:
train_df = pd.read_csv(config.TRAIN_CSV)
train_df.head(10)

In [ ]:
def image_path_creation(path, replace_part):
    file_names = []
    full_paths = []
    for dirpath,  dirname, filenames in os.walk(path):
        if filenames:
            for filename in filenames:
                full_path = os.path.join(dirpath, filename)
                filename = filename.replace(replace_part, '')
                full_paths.append(full_path)
                file_names.append(filename)
    return file_names, full_paths

In [ ]:
file_names, full_paths = image_path_creation(config.TRAIN_PATH, '-c.dcm')
image_path_dict = {'SOPInstanceUID': file_names, 'path':full_paths}
image_df = pd.DataFrame(image_path_dict)

In [ ]:
train_df = pd.merge(train_df, image_df, on = 'SOPInstanceUID')

In [ ]:
train_df.head(8)

In [ ]:
train_df.shape

<a id="2"></a>

# <span><center style = "font-family:Verdana; font-size:36px; letter-spacing:2px; font-weight:bold; color:teal">2.&nbsp;&nbsp;CONVERTING IMAGE DATA TO JPG&nbsp;&nbsp;&nbsp;&nbsp;<a href='#toc'>&#10514;</a></center></span>

In [ ]:
def read_dicom_image(path):
    dicom_file = dcmread(path)
    image = dicom_file.pixel_array
    image = image/image.max()
    return image

In [ ]:
def save_image_jpg(path, save_path_name):
    image = read_dicom_image(path)*255
    plt.imsave(save_path_name, image)

In [ ]:
os.listdir()

In [ ]:
# shutil.rmtree('./train')

In [ ]:
# Saving jpg images to working directory

# os.mkdir('./train')

for i in tqdm(range(1000, train_df.shape[0])):
    save_image_jpg(train_df.path[i], f'./train/{train_df.SOPInstanceUID[i]}.jpg')

In [ ]:
file_names, file_paths = image_path_creation(config.TRAIN_JPG_PATH, '.jpg')
image_df = {'SOPInstanceUID': file_names, 'path_jpg':file_paths}
image_df = pd.DataFrame(image_df)

In [ ]:
os.listdir(config.TRAIN_JPG_PATH)

In [ ]:
image_df#, train_df.SOPInstanceUID[0]

In [ ]:
train_df_jpg=pd.merge(train_df, image_df, on = 'SOPInstanceUID')

In [ ]:
train_df_jpg.head()

In [ ]:
fig = plt.figure(figsize=(25,15))
for i in range(15):
    ax = fig.add_subplot(3,5, i+1)
#     image = read_dicom_image(train_df.path_jpg[i*5])
    img = np.array(Image.open(train_df_jpg.path_jpg[i*5]))
    plt.imshow(img, cmap = 'gray')
    ax.set_title(train_df.Target[i*5])


In [ ]:
img = img/255

In [ ]:
cv2.ConvertColor(img, )